<a href="https://colab.research.google.com/github/emmanuelvaie/google_colab/blob/main/licence2cahier_electronique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds) 

In [99]:
worksheet_licence = gc.open_by_url('https://docs.google.com/spreadsheets/d/15vnCNwatQkb-Ag4K6jvIL1pXpZRwtBrhcbCJ6397LDA/edit#gid=806761863').sheet1

# get_all_values gives a list of rows.
rows_licences = worksheet_licence.get_all_values()

import pandas as pd
df_licences = pd.DataFrame.from_records(rows_licences,columns=rows_licences[0])
df_licences = df_licences.drop(0).reindex()

In [100]:
df_licences.count()

NO ADHERENT                 197
NOM                         197
PRENOM                      197
DATE NAISSANCE              197
SEXE                        197
DIFFUSION ADRESSE           197
CATEGORIE LICENCE           197
ADRESSE 1                   197
ADRESSE 2                   197
ADRESSE 3                   197
ADRESSE 4                   197
CODE POSTAL                 197
VILLE                       197
PAYS                        197
N TEL                       197
EMAIL                       197
CODE INSEE                  197
NO STR                      197
LIGUE                       197
CD                          197
DATE DE SAISIE              197
PORTEUR D'UN HANDICAP       197
TYPE DE HANDICAP            197
CLASSIFICATION ENVISAGEE    197
CLASSIFICATION              197
SOUS-CLASSIFICATION         197
CLASSIFICATION VI           197
NIVEAU CLASSIFICATION       197
MODE SAISIE                 197
dtype: int64

In [101]:
df_licences.columns

Index(['NO ADHERENT', 'NOM', 'PRENOM', 'DATE NAISSANCE', 'SEXE',
       'DIFFUSION ADRESSE', 'CATEGORIE LICENCE', 'ADRESSE 1', 'ADRESSE 2',
       'ADRESSE 3', 'ADRESSE 4', 'CODE POSTAL', 'VILLE', 'PAYS', 'N TEL',
       'EMAIL', 'CODE INSEE', 'NO STR', 'LIGUE', 'CD', 'DATE DE SAISIE',
       'PORTEUR D'UN HANDICAP', 'TYPE DE HANDICAP', 'CLASSIFICATION ENVISAGEE',
       'CLASSIFICATION', 'SOUS-CLASSIFICATION', 'CLASSIFICATION VI',
       'NIVEAU CLASSIFICATION', 'MODE SAISIE'],
      dtype='object')

In [102]:
df_licences = df_licences.drop(columns=['DIFFUSION ADRESSE','ADRESSE 1', 'ADRESSE 2',
       'ADRESSE 3', 'ADRESSE 4','N TEL',
       'EMAIL', 'CODE INSEE', 'NO STR', 'LIGUE', 'CD', 'DATE DE SAISIE',
       'PORTEUR D\'UN HANDICAP', 'TYPE DE HANDICAP', 'CLASSIFICATION ENVISAGEE',
       'CLASSIFICATION', 'SOUS-CLASSIFICATION', 'CLASSIFICATION VI',
       'NIVEAU CLASSIFICATION', 'MODE SAISIE'])

In [103]:
df_licences = df_licences.rename(columns={'PRENOM':'FirstName', 'NOM':'LastName',\
                                          'DATE NAISSANCE':'Birthday','SEXE':'Gender',\
                            'CATEGORIE LICENCE':'StatusId','VILLE':'AddressCity',\
                            'CODE POSTAL':'AddressZip',\
                            'NO ADHERENT':'MembershipNo','PAYS':'AddressCountry'})

In [104]:
df_licences['Gender'] = df_licences['Gender'].apply(lambda x : 'féminin' if x=='F' else 'masculin')
df_licences['Birthday'] = df_licences['Birthday'].apply(lambda x : str(x).replace('/','.'))

In [105]:
df_licences['StatusId'] = df_licences['StatusId'].apply(lambda x : 'Sénior' if x=='A' else 'Scolaire')

In [106]:
df_licences['Association'] = 'SNBS'
df_licences['AddressCountry'] = 'FRANCE'
#df_licences['NameAffix'] = ''
#df_licences['Title'] = ''

In [107]:
first_column = df_licences.pop('FirstName')
df_licences.insert(0, 'FirstName', first_column)

In [109]:
col = df_licences.pop('LastName')
df_licences.insert(1, 'LastName', col)
#col = df_licences.pop('NameAffix')
#df_licences.insert(2, 'NameAffix', col)
#col = df_licences.pop('Title')
#df_licences.insert(3, 'Title', col)
col = df_licences.pop('Gender')
df_licences.insert(4, 'Gender', col)
col = df_licences.pop('Birthday')
df_licences.insert(5, 'Birthday', col)
col = df_licences.pop('Association')
df_licences.insert(6, 'Association', col)
col = df_licences.pop('StatusId')
df_licences.insert(7, 'StatusId', col)


In [110]:
df_licences = df_licences.sort_values(by=['LastName'])

In [111]:
df_licences.head(5)

,FirstName,LastName,MembershipNo,Gender,Birthday,Association,AddressZip,StatusId,AddressCity,AddressCountry
97,OLIVIER,ALEXANDRE,44215,masculin,28.05.1958,SNBS,92270,Sénior,BOIS COLOMBES,FRANCE
134,DAPHNE,ANDRE,539073,féminin,28.06.2009,SNBS,92400,Sénior,COURBEVOIE,FRANCE
192,ELIOTT,BAKER,568377,masculin,26.10.2009,SNBS,92400,Sénior,COURBEVOIE,FRANCE
172,EMMANUEL,BAPT,566419,masculin,05.02.1968,SNBS,92200,Sénior,NEUILLY SUR SEINE,FRANCE
107,YOHAN,BARA-BUTEZ,467324,masculin,22.06.2003,SNBS,92250,Sénior,LA GARENNE COLOMBES,FRANCE


In [112]:
nb_rows = df_licences['MembershipNo'].count()
nb_cols = len(df_licences.columns)

In [114]:
try :
  licences_reprocessed = worksheet_licence.spreadsheet.worksheet('EFA')
  licences_reprocessed.clear()
except:
  licences_reprocessed = worksheet_licence.spreadsheet.add_worksheet(title='EFA', rows=nb_rows, cols=nb_cols)

from gspread_dataframe import set_with_dataframe
set_with_dataframe(licences_reprocessed, df_licences)